In [41]:
# Load packages
import pyam
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Optional
from typing import List, Optional, Union
import math as math
import warnings
import os
import pathlib
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
from typing import List, Optional, Dict

import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker
import pathlib, math
from typing import List, Dict, Optional


# Only in interactive/Jupyter contexts:
%matplotlib inline

# Ignore all warnings
warnings.filterwarnings("ignore")


# 1) Load data

In [42]:
# -------------------
# Function to load and concatenate files from a directory
# -------------------

def load_and_concat_files_then_pyam(directory_path, file_types=None):
    """
    Load specified file types from a directory, concatenate them as pandas DataFrames,
    then convert the result to a pyam IamDataFrame.
    
    Parameters:
    -----------
    directory_path : str
        Path to the directory containing data files
    file_types : str or list, optional
        Specify which file types to process: "xlsx", "csv", or ["xlsx", "csv"]
        If None, both xlsx and csv files will be processed
    
    Returns:
    --------
    pyam.IamDataFrame
        Combined IamDataFrame from all files
    """
    # Set default file types if not specified
    if file_types is None:
        file_types = ["xlsx", "csv"]
    elif isinstance(file_types, str):
        file_types = [file_types]
    
    # Create lookup for file extensions
    valid_extensions = {
        "xlsx": ".xlsx",
        "csv": ".csv"
    }
    
    # Get the extensions to look for
    extensions_to_check = [valid_extensions[ft] for ft in file_types if ft in valid_extensions]
    
    # List to store all pandas dataframes
    all_dfs = []
    
    # Get all files with the specified extensions in the directory
    for file_name in os.listdir(directory_path):
        if any(file_name.endswith(ext) for ext in extensions_to_check):
            full_path = os.path.join(directory_path, file_name)
            
            try:
                # Read the file based on its extension
                if file_name.endswith('.csv'):
                    try:
                        data_file = pd.read_csv(full_path)
                    except UnicodeDecodeError:
                        data_file = pd.read_csv(full_path, encoding='ISO-8859-1')
                elif file_name.endswith('.xlsx'):
                    data_file = pd.read_excel(full_path)
                
                # Add to list of pandas DataFrames
                all_dfs.append(data_file)
                print(f"Read {file_name}")
                
            # If there is an error, print a message and continue to the next file
            except Exception as e:
                print(f"Error reading {file_name}: {e}")
                continue
    
    # Concatenate all pandas dataframes
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        # Convert the combined DataFrame to pyam IamDataFrame
        result_df = pyam.IamDataFrame(combined_df)
        return result_df
    else:
        print("No files were processed")
        return None

In [43]:
# Load data
path = "../data/"  
df = load_and_concat_files_then_pyam(path, file_types=["csv"])

Read DACCSProj_standardized.csv
Read energy system cost_TIAM_GCAM.csv
Read mastersheet_dacc_derp_26052025.csv


In [44]:
df

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : GCAM 7.0, TIAM_Grantham (2)
 * scenario : D2_NDC_DACCS_2_4, D3_HP_PriceShock, D3_NG_PriceShock, D5_NG_HP_PriceShock (4)
Timeseries data coordinates:
   region   : AFR, AUS, Africa_Eastern, Africa_Northern, ... World (47)
   variable : Agricultural Demand, Agricultural Demand|Crops, ... energy-service (492)
   unit     : EJ/yr, GW, GW/yr, Index (2005 = 1), M$(2010)/yr, ... ºC (44)
   year     : 2005, 2010, 2015, 2020, 2025, 2030, 2035, 2040, ... 2100 (20)

# 2) Explore and modify the data

In [45]:
# -------------------
# Function to explore data
# -------------------

def explore_data(iam_df: pyam.IamDataFrame):
    """Explores a pyam.IamDataFrame (lists regions, variables, models, scenarios)."""

    if iam_df is None:
        print("Error: No IamDataFrame provided for exploration.")
        return

    print("\nAvailable Regions:", iam_df.region)
    print("\nAvailable Variables:", iam_df.variable)
    print("\nAvailable Models:", iam_df.model)
    print("\nAvailable Scenarios:", iam_df.scenario)

# -------------------
# Examples
# -------------------
# # Explore the combined data
# explore_data(all_data)


In [46]:
explore_data(df)


Available Regions: ['AFR', 'AUS', 'Africa_Eastern', 'Africa_Northern', 'Africa_Southern', 'Africa_Western', 'Argentina', 'Australia_NZ', 'Brazil', 'CAN', 'CHI', 'CSA', 'Canada', 'Central America and Caribbean', 'Central Asia', 'China', 'Colombia', 'EEU', 'EU-12', 'EU-15', 'Europe_Eastern', 'Europe_Non_EU', 'European Free Trade Association', 'FSU', 'IND', 'India', 'Indonesia', 'JPN', 'Japan', 'MEA', 'MEX', 'Mexico', 'Middle East', 'ODA', 'Pakistan', 'Russia', 'SKO', 'South Africa', 'South America_Northern', 'South America_Southern', 'South Asia', 'South Korea', 'Southeast Asia', 'Taiwan', 'USA', 'WEU', 'World']

Available Variables: ['Agricultural Demand', 'Agricultural Demand|Crops', 'Agricultural Demand|Crops|Energy', 'Agricultural Demand|Crops|Feed', 'Agricultural Demand|Crops|Food', 'Agricultural Demand|Crops|Other', 'Agricultural Demand|Livestock', 'Agricultural Demand|Livestock|Food', 'Agricultural Demand|Livestock|Other', 'Agricultural Production', 'Capacity Additions|Electricit

In [47]:
# -------------------
# Function to rename data
# -------------------

def rename_data(iam_df: pyam.IamDataFrame, renaming: dict, obj_type: str = "scenario") -> Optional[pyam.IamDataFrame]:
    """Renames scenarios, models, or variables in a pyam.IamDataFrame."""

    if iam_df is None:
        print("Error: No IamDataFrame provided for renaming.")
        return None

    try:
        iam_df.rename(**{obj_type: renaming}, inplace=True)
        return iam_df
    except Exception as e:
        print(f"Error during renaming: {e}")
        return None

# -------------------
# Examples
# -------------------

# # Rename scenarios
# scenario_renaming = {
#     'DERP1': 'NDC_EI_DERP1',
#     #... (rest of your renaming mapping)
# }
# renamed_data = rename_data(all_data, scenario_renaming)

# # Rename Models
# model_renaming = {
#     'AIM/CGE': 'AIM',
#     'MESSAGEix-GLOBIOM 1.0': 'MESSAGE',
# }
# renamed_models = rename_data(all_data, model_renaming, obj_type='model')

In [48]:
# Add up energy-service and energy-investment and create a new var called 'Energy System Cost' in GCAM

df.add(
    "energy-investment",   # input side (e.g. capital cost category)
    "energy-service",      # output side (e.g. what this investment enables)
    name="Energy System Cost",   # label for reporting
    ignore_units=True,           # bypass unit consistency checks
    append=True,                 # keep existing definitions, just add this one
#     unit="M$(2010)/yr"           # reporting unit

)


In [49]:
df.rename(unit={'unknown': 'M$(2010)/yr'}, inplace=True)


In [50]:
df.filter(model='GCAM 7.0').filter(variable='Energy System Cost').timeseries()

2005  \
model    scenario            region          variable           unit                      
GCAM 7.0 D2_NDC_DACCS_2_4    Africa_Eastern  Energy System Cost M$(2010)/yr     57662.0   
                             Africa_Northern Energy System Cost M$(2010)/yr    219266.0   
                             Africa_Southern Energy System Cost M$(2010)/yr     51240.0   
                             Africa_Western  Energy System Cost M$(2010)/yr    139743.0   
                             Argentina       Energy System Cost M$(2010)/yr    151353.0   
...                                                                                 ...   
         D5_NG_HP_PriceShock South Korea     Energy System Cost M$(2010)/yr    620029.0   
                             Southeast Asia  Energy System Cost M$(2010)/yr    449243.0   
                             Taiwan          Energy System Cost M$(2010)/yr    130000.0   
                             USA             Energy System Cost M$(2010)/yr   3969557.0   
                             World           Energy System Cost M$(2010)/yr  19071414.0   

                                                                                   2010  \
model    scenario            region          variable           unit                      
GCAM 7.0 D2_NDC_DACCS_2_4    Africa_Eastern  Energy System Cost M$(2010)/yr     87532.0   
                             Africa_Northern Energy System Cost M$(2010)/yr    335332.0   
                             Africa_Southern Energy System Cost M$(2010)/yr     76451.0   
                             Africa_Western  Energy System Cost M$(2010)/yr    207205.0   
                             Argentina       Energy System Cost M$(2010)/yr    186630.0   
...                                                                                 ...   
         D5_NG_HP_PriceShock South Korea     Energy System Cost M$(2010)/yr    706299.0   
                             Southeast Asia  Energy System Cost M$(2010)/yr    624626.0   
                             Taiwan          Energy System Cost M$(2010)/yr    139980.0   
                             USA             Energy System Cost M$(2010)/yr   4258959.0   
                             World           Energy System Cost M$(2010)/yr  23337008.0   

                                                                                   2020  \
model    scenario            region          variable           unit                      
GCAM 7.0 D2_NDC_DACCS_2_4    Africa_Eastern  Energy System Cost M$(2010)/yr    161544.0   
                             Africa_Northern Energy System Cost M$(2010)/yr    516735.0   
                             Africa_Southern Energy System Cost M$(2010)/yr    133947.0   
                             Africa_Western  Energy System Cost M$(2010)/yr    324078.0   
                             Argentina       Energy System Cost M$(2010)/yr    330451.0   
...                                                                                 ...   
         D5_NG_HP_PriceShock South Korea     Energy System Cost M$(2010)/yr   1011284.0   
                             Southeast Asia  Energy System Cost M$(2010)/yr    982528.0   
                             Taiwan          Energy System Cost M$(2010)/yr    175212.0   
                             USA             Energy System Cost M$(2010)/yr   5100995.0   
                             World           Energy System Cost M$(2010)/yr  30553601.0   

                                                                                   2030  \
model    scenario            region          variable           unit                      
GCAM 7.0 D2_NDC_DACCS_2_4    Africa_Eastern  Energy System Cost M$(2010)/yr    254414.0   
                             Africa_Northern Energy System Cost M$(2010)/yr    819534.0   
                             Africa_Southern Energy System Cost M$(2010)/yr    233832.0   
                             Africa_Western  Energy System Cost M$(2010)/yr 

# 3) Explore variables

In [51]:
### Variables groups ###

# Capacity Additions (stacked area plot)
capacity_additions = [
    # 'Capacity Additions|Electricity|Biomass',
    # 'Capacity Additions|Electricity|Fossil',
    # 'Capacity Additions|Electricity|Coal',
    # 'Capacity Additions|Electricity|Gas',
    # 'Capacity Additions|Electricity|Oil',
    # 'Capacity Additions|Electricity|Biomass|w/ CCS',
    # 'Capacity Additions|Electricity|Biomass|w/o CCS',
    # 'Capacity Additions|Electricity|Coal|w/ CCS',
    # 'Capacity Additions|Electricity|Coal|w/o CCS',
    # 'Capacity Additions|Electricity|Gas|w/ CCS',
    # 'Capacity Additions|Electricity|Gas|w/o CCS',
    # 'Capacity Additions|Electricity|Oil|w/ CCS',
    # 'Capacity Additions|Electricity|Oil|w/o CCS',
    # 'Capacity Additions|Electricity|Solar',
    # 'Capacity Additions|Electricity|Wind',
    # 'Capacity Additions|Electricity|Geothermal',
    # 'Capacity Additions|Electricity|Nuclear',
    # 'Capacity Additions|Electricity|Hydro',
]

# Overall capacity (stacked area plot)
overall_capacity_electricity = [
    # 'Capacity|Electricity|Biomass',
    # 'Capacity|Electricity|Biomass|w/ CCS',
    # 'Capacity|Electricity|Biomass|w/o CCS',
    # # 'Capacity|Electricity|Coal',
    # 'Capacity|Electricity|Coal|w/ CCS',  # Might need renaming!
    # 'Capacity|Electricity|Coal|w/o CCS',
    # # 'Capacity|Electricty|Coal|CCS',
    # # 'Capacity|Electricity|Gas',
    # 'Capacity|Electricity|Gas|w/ CCS',
    # 'Capacity|Electricity|Gas|w/o CCS',
    # # 'Capacity|Electricity|Gas|CCS',
    # # 'Capacity|Electricity|Oil',
    # 'Capacity|Electricity|Oil|w/ CCS',
    # 'Capacity|Electricity|Oil|w/o CCS',
    # 'Capacity|Electricity|Solar',
    # # 'Capacity|Electricity|Solar|CSP',
    # # 'Capacity|Electricity|Solar|PV',
    # 'Capacity|Electricity|Wind',
    # # 'Capacity|Electricity|Wind|Offshore',
    # # 'Capacity|Electricity|Wind|Onshore',
    # 'Capacity|Electricity|Geothermal',
    # 'Capacity|Electricity|Hydro',
    # 'Capacity|Electricity|Nuclear',
    # 'Capacity|Electricity|Ocean',
]

# Emissions (linear individual plots)
emissions = [
    # 'Emissions|CO2',
    # 'Emissions|CH4',
    # 'Emissions|Kyoto Gases',
    # 'Emissions|N2O',
    # 'Emissions|NOx',
    # 'Emissions|BC',
    # 'Emissions|CF4',
    # 'Emissions|C2F6',
    # 'Emissions|CO',
    # 'Emissions|F-Gases',
    # 'Emissions|HFC',
    # 'Emissions|NH3',
    # 'Emissions|OC',
    # 'Emissions|SF6',
    # 'Emissions|Sulfur',
    # 'Emissions|VOC',
]

# CO2 emissions (stacked area plot)
CO2_emissions = [
    # 'Emissions|CO2|AFOLU',
    # 'Emissions|CO2|Energy',
    # 'Emissions|CO2|Energy and Industrial Processes',
    # 'Emissions|CO2|Fossil Fuels and Industry',
    # 'Emissions|CO2|Industrial Processes',
    # 'Emissions|CO2|Product Use',
    # 'Emissions|CO2|Waste',
    # 'Emissions|CO2|Capture and Removal',  # You might want to uncomment this if relevant
]

# carbon sequestration (stacked area plot)
carbon_sequestration = [
    'Carbon Sequestration|Direct Air Capture',
#  'Carbon Sequestration|CCS',
    'Carbon Sequestration|CCS|Biomass',
#  'Carbon Sequestration|CCS|Biomass|Energy|Supply',
#  'Carbon Sequestration|CCS|Biomass|Energy|Supply|Electricity',
#  'Carbon Sequestration|CCS|Biomass|Energy|Supply|Liquids',
#  'Carbon Sequestration|CCS|Biomass|Energy|Supply|Hydrogen'
#  'Carbon Sequestration|CCS|Biomass|Energy|Demand|Industry',
    'Carbon Sequestration|CCS|Fossil',
    'Carbon Sequestration|CCS|Fossil|Energy|Demand|Industry',
    'Carbon Sequestration|CCS|Fossil|Energy|Supply',
#  'Carbon Sequestration|CCS|Fossil|Energy|Supply|Electricity',
#  'Carbon Sequestration|CCS|Fossil|Energy|Supply|Hydrogen',
#  'Carbon Sequestration|CCS|Fossil|Energy|Supply|Liquids',
#  'Carbon Sequestration|CCS|Industrial Processes',
    # 'Carbon Sequestration|CCS|Direct Air Capture',
    # 'Carbon Sequestration|CCS|Industrial Processes',
    # 'Carbon Sequestration|Land Use',
    # 'Carbon Sequestration|CCS|Industry',

]

# Land use sequestration (stacked area plot)
land_use_sequestration = [
#     'Carbon Sequestration|Land Use|Afforestation',
#     'Carbon Sequestration|Land Use|Agriculture',
#     'Carbon Sequestration|Land Use|Forest Management',
#     'Carbon Sequestration|Land Use|Other',
#     'Carbon Sequestration|Land Use|Other LUC',
]

# Carbon dioxide removal (stacked area plot)
carbon_dioxide_removal = [
    # 'Carbon Sequestration|Direct Air Capture',
    # 'Carbon Sequestration|Ocean',
    # 'Carbon Sequestration|Soil Carbon',
    # 'Carbon Sequestration|Land Use|Afforestation',
    # 'Carbon Sequestration|Land Use|Agriculture',
    # 'Carbon Sequestration|Land Use|Forest Management',
    # 'Carbon Sequestration|Land Use|Other',
    # 'Carbon Sequestration|Land Use|Other LUC',   
    # 'Carbon Removal',
    # 'Carbon Removal|Geological Storage|Direct Air Capture',
    # # 'Carbon Removal|Geological Storage',
    # 'Carbon Removal|Geological Storage|Biomass',
    # 'Carbon Removal|Land Use',
    # 'Carbon Capture|Geological Storage|Direct Air Capture',
    # # 'Carbon Sequestration|Geological Storage',
    # 'Carbon Capture|Geological Storage|Other Sources',
    # 'Carbon Capture|Industrial Processes',
#     'Carbon Removal|Geological Storage',
#  'Carbon Removal|Land Use',
#  'Gross Removals|CO2'
]

# Overall picture (linear individual plots)
timeseries_variables = [
    'Carbon Sequestration|CCS',
    # 'Carbon Sequestration|Land Use',
    # 'Primary Energy',
    'Final Energy',
    # 'GDP|PPP',
    'Emissions|CO2',
    # 'Temperature|Global Mean',
    # 'Investment|Energy Supply|Electricity',
    # 'Land Cover',
    # 'Secondary Energy',
    # 'Capacity Additions|Electricity',
    'Capacity|Electricity',
    # 'Price|Carbon',
    # 'CO2|Energy and Industrial Processes',
    'Emissions|CO2|Energy and Industrial Processes',
    'Final Energy|Electricity',
    'Secondary Energy|Electricity',
    'Carbon Sequestration|Direct Air Capture',

    'Primary Energy|Oil',
    'Primary Energy|Non-Biomass Renewables',
    'Primary Energy|Geothermal',
    'Primary Energy|Hydro',
    'Primary Energy|Nuclear',
    # 'Primary Energy|Ocean',
    'Primary Energy|Solar',
    'Primary Energy|Wind',

  'Primary Energy',
 'Primary Energy|Biomass',
#  'Primary Energy|Biomass|1st Generation',
#  'Primary Energy|Biomass|Modern',
#  'Primary Energy|Biomass|Traditional',
 'Primary Energy|Coal',
#  'Primary Energy|Coal|w/ CCS',
#  'Primary Energy|Coal|w/o CCS',
 'Primary Energy|Fossil',
#  'Primary Energy|Fossil|w/ CCS',
#  'Primary Energy|Fossil|w/o CCS',
 'Primary Energy|Gas',
#  'Primary Energy|Gas|w/ CCS',
#  'Primary Energy|Gas|w/o CCS',
 'Primary Energy|Non-Biomass Renewables|Geothermal',
 'Primary Energy|Non-Biomass Renewables|Hydro',
 'Primary Energy|Non-Biomass Renewables|Solar',
 'Primary Energy|Non-Biomass Renewables|Wind',
]

# Investment in energy supply (stacked area plot)
energy_investment = [
    # 'Investment|Energy Supply|CO2 Transport and Storage',
    # 'Investment|Energy Supply|Electricity',
    # 'Investment|Energy Supply|Heat',
    # 'Investment|Energy Supply|Hydrogen',
    # 'Investment|Energy Supply|Liquids',
    # 'Investment|Energy Supply|Other',
    # 'Investment|Energy Supply|Electricity|Biomass|w/ CCS',
    # 'Investment|Energy Supply|Electricity|Biomass|w/o CCS',
    # 'Investment|Energy Supply|Electricity|Coal|w/ CCS',
    # 'Investment|Energy Supply|Electricity|Coal|w/o CCS',
    # 'Investment|Energy Supply|Electricity|Gas|w/ CCS',
    # 'Investment|Energy Supply|Electricity|Gas|w/o CCS',
    # 'Investment|Energy Supply|Electricity|Oil|w/ CCS',
    # 'Investment|Energy Supply|Electricity|Oil|w/o CCS',
    # 'Investment|Energy Supply|Electricity|Solar',
    # 'Investment|Energy Supply|Electricity|Wind',
    # 'Investment|Energy Supply|Electricity|Geothermal',
    # 'Investment|Energy Supply|Electricity|Hydro',
    # 'Investment|Energy Supply|Electricity|Nuclear',
    # 'Investment|Energy Supply|Electricity|Transmission and Distribution',
]

# Primary energy (stacked area plot)
primary_energy = [
    # 'Primary Energy|Biomass',
    # 'Primary Energy|Biomass|w/ CCS',
    # 'Primary Energy|Biomass|w/o CCS',
    # 'Primary Energy|Coal',
    # 'Primary Energy|Coal|w/ CCS',
    # 'Primary Energy|Coal|w/o CCS',
    # # 'Primary Energy|Fossil',  
    # 'Primary Energy|Gas',
    # 'Primary Energy|Gas|w/ CCS',
    # 'Primary Energy|Gas|w/o CCS',
    # 'Primary Energy|Geothermal',
    # 'Primary Energy|Hydro',
    # # 'Primary Energy|Non-Biomass Renewables',  
    # 'Primary Energy|Nuclear',
    # 'Primary Energy|Ocean',
    # 'Primary Energy|Oil',
    # 'Primary Energy|Oil|w/ CCS',
    # 'Primary Energy|Oil|w/o CCS',
    # 'Primary Energy|Other',
    # 'Primary Energy|Secondary Energy Trade',
    # 'Primary Energy|Solar',
    # 'Primary Energy|Wind',
    # 'Primary Energy|Non-Biomass Renewables',
    # 'Primary Energy|Other',
    
    # 'Primary Energy|Oil',
    'Primary Energy|Non-Biomass Renewables',
    # 'Primary Energy|Geothermal',
    # 'Primary Energy|Hydro',
    # 'Primary Energy|Nuclear',
    # # 'Primary Energy|Ocean',
    # 'Primary Energy|Solar',
    # 'Primary Energy|Wind',

#   'Primary Energy',
 'Primary Energy|Biomass',
# #  'Primary Energy|Biomass|1st Generation',
# #  'Primary Energy|Biomass|Modern',
# #  'Primary Energy|Biomass|Traditional',
#  'Primary Energy|Coal',
# #  'Primary Energy|Coal|w/ CCS',
# #  'Primary Energy|Coal|w/o CCS',
 'Primary Energy|Fossil',
#  'Primary Energy|Fossil|w/ CCS',
#  'Primary Energy|Fossil|w/o CCS',
#  'Primary Energy|Gas',
# #  'Primary Energy|Gas|w/ CCS',
# #  'Primary Energy|Gas|w/o CCS',
#  'Primary Energy|Non-Biomass Renewables|Geothermal',
#  'Primary Energy|Non-Biomass Renewables|Hydro',
#  'Primary Energy|Non-Biomass Renewables|Solar',
#  'Primary Energy|Non-Biomass Renewables|Wind',
]


# Final energy by carriers (stacked area plot)
final_energy_by_carriers = [
    'Final Energy|Electricity',
    'Final Energy|Gases',
    'Final Energy|Heat',
    'Final Energy|Liquids',
    'Final Energy|Solids',
    # 'Final Energy|Solids|Biomass', 
    #  'Final Energy|Solids|Biomass|Traditional',
    # 'Final Energy|Solids|Coal', 
    # 'Final Energy|Other Sector',
]

# Final energy by sources (stacked area plot)
final_energy_by_sources = [
    'Final Energy|Geothermal',
    'Final Energy|Solar',
    'Final Energy|Hydrogen',
]

# Finals by sector (stacked area plot)
finals_by_sector = [
    # 'Final Energy|Agriculture', 
    # 'Final Energy|Bunkers', 
    # 'Final Energy|Commercial', 
    'Final Energy|Industry',
    'Final Energy|Non-Energy Use', 
    # 'Final Energy|Residential', 
    'Final Energy|Residential and Commercial',
    'Final Energy|Transportation',
    # 'Final Energy|Transportation (w/ bunkers)', 
    # 'Final Energy|Other Sector', 
    # 'Final Energy|Other Sector|Electricity',
    # 'Final Energy|Non-Energy Use', 
    # 'Final Energy|Non-Energy Use|Coal',
    # 'Final Energy|Non-Energy Use|Gas',
    # 'Final Energy|Non-Energy Use|Oil',
    # 'Final Energy|Non-Energy Use|Biomass',
]

finals_by_sector_electricity = [
    'Final Energy|Industry|Electricity',
    'Final Energy|Residential and Commercial|Electricity',
    'Final Energy|Transportation|Electricity',
]

# Secondary energy (stacked area plot)
secondary_energy_electricity = [
    'Secondary Energy|Electricity|Biomass',
    # 'Secondary Energy|Electricity|Biomass|w/ CCS', 
    # 'Secondary Energy|Electricity|Biomass|w/o CCS', 
    'Secondary Energy|Electricity|Coal',
    # 'Secondary Energy|Electricity|Coal|w/ CCS', 
    # 'Secondary Energy|Electricity|Coal|w/o CCS', 
    'Secondary Energy|Electricity|Gas',
    # 'Secondary Energy|Electricity|Gas|w/ CCS', 
    # 'Secondary Energy|Electricity|Gas|w/o CCS', 
    'Secondary Energy|Electricity|Oil',
    # 'Secondary Energy|Electricity|Oil|w/o CCS', 
    'Secondary Energy|Electricity|Geothermal',
    'Secondary Energy|Electricity|Hydro',
    'Secondary Energy|Electricity|Nuclear',
    'Secondary Energy|Electricity|Solar',
    'Secondary Energy|Electricity|Wind',
    # 'Secondary Energy|Electricity|Non-Biomass Renewables', 
    # 'Secondary Energy|Gases', 
    # 'Secondary Energy|Heat', 
    # 'Secondary Energy|Hydrogen', 
    # 'Secondary Energy|Liquids', 
    # 'Secondary Energy|Solids', 

    'Secondary Energy|Electricity|Ocean',
    # 'Secondary Energy|Electricity|Solar|CSP',
    # 'Secondary Energy|Electricity|Solar|PV',
    # 'Secondary Energy|Electricity|Wind|Offshore',
    # 'Secondary Energy|Electricity|Wind|Onshore',


    # 'Secondary Energy|Hydrogen|Fossil',
    # 'Secondary Energy|Hydrogen|Fossil|w/ CCS',
    # 'Secondary Energy|Hydrogen|Fossil|w/o CCS',
    # 'Secondary Energy|Liquids|Oil',
    # 'Secondary Energy|Liquids|Biomass',
    # 'Secondary Energy|Liquids|Biomass|w/ CCS',
    # 'Secondary Energy|Liquids|Gas',
    # 'Secondary Energy|Liquids|Gas|w/o CCS',
    # 'Secondary Energy|Hydrogen|Biomass',
    # 'Secondary Energy|Hydrogen|Biomass|w/ CCS',
    # 'Secondary Energy|Liquids|Biomass|w/o CCS',
    # 'Secondary Energy|Hydrogen|Electricity',

    #  'Secondary Energy|Gases|Biomass',
    #  'Secondary Energy|Gases|Coal',
    #  'Secondary Energy|Gases|Natural Gas',
    #  'Secondary Energy|Hydrogen|Biomass|w/o CCS',
    #  'Secondary Energy|Hydrogen|Coal',
    #  'Secondary Energy|Hydrogen|Coal|w/ CCS',
    #  'Secondary Energy|Hydrogen|Gas',
    #  'Secondary Energy|Hydrogen|Gas|w/ CCS',
    #  'Secondary Energy|Hydrogen|Gas|w/o CCS',
    #  'Secondary Energy|Liquids|Biomass|w/o CCS',
    #  'Secondary Energy|Liquids|Coal',
    #  'Secondary Energy|Liquids|Coal|w/ CCS',
    #  'Secondary Energy|Liquids|Coal|w/o CCS',
    #  'Secondary Energy|Liquids|Fossil',
    #  'Secondary Energy|Liquids|Fossil|w/ CCS',
    #  'Secondary Energy|Liquids|Fossil|w/o CCS',
    #  'Secondary Energy|Solids|Biomass',
    #  'Secondary Energy|Solids|Coal',
]

secondary_energy_non_electric = [
        'Secondary Energy|Gases', # Added, uncommented
    'Secondary Energy|Hydrogen', # Added, uncommented
    # 'Secondary Energy|Hydrogen|Electricity',
    # 'Secondary Energy|Hydrogen|Fossil',
    # 'Secondary Energy|Hydrogen|Fossil|w/ CCS',
    # 'Secondary Energy|Hydrogen|Fossil|w/o CCS',
    'Secondary Energy|Liquids', # Added, uncommented
    # 'Secondary Energy|Liquids|Biomass',
    # 'Secondary Energy|Liquids|Biomass|w/ CCS',
    # 'Secondary Energy|Liquids|Biomass|w/o CCS',
    # 'Secondary Energy|Liquids|Gas',
    # 'Secondary Energy|Liquids|Gas|w/o CCS',
    # 'Secondary Energy|Liquids|Oil',
    'Secondary Energy|Solids', # Added, uncommented
]


# Residential final energy (stacked area plot)
residential_final_energy = [
    # 'Final Energy|Residential and Commercial|Residential|Cooling', 
    # 'Final Energy|Residential and Commercial|Residential|Electricity', 
    # 'Final Energy|Residential and Commercial|Residential|Gases', 
    # 'Final Energy|Residential and Commercial|Residential|Heat', 
    # 'Final Energy|Residential and Commercial|Residential|Heating|Space', 
    # 'Final Energy|Residential and Commercial|Residential|Hydrogen', 
    # 'Final Energy|Residential and Commercial|Residential|Liquids', 
#  'Final Energy|Residential and Commercial|Commercial|Solids',
#  'Final Energy|Residential and Commercial|Commercial|Solids|Biomass',
#  'Final Energy|Residential and Commercial|Commercial|Solids|Coal',
]

# Commercial final energy (stacked area plot)
commercial_final_energy = [
    # 'Final Energy|Residential and Commercial|Commercial|Cooling', 
    # 'Final Energy|Residential and Commercial|Commercial|Electricity', 
    # 'Final Energy|Residential and Commercial|Commercial|Gases', 
    # 'Final Energy|Residential and Commercial|Commercial|Heat', 
    # 'Final Energy|Residential and Commercial|Commercial|Heating|Space', 
    # 'Final Energy|Residential and Commercial|Commercial|Hydrogen', 
    # 'Final Energy|Residential and Commercial|Commercial|Liquids', 
    # 'Final Energy|Residential and Commercial|Commercial|Solids', 
    # 'Final Energy|Residential and Commercial|Commercial|Solids|Biomass', 
    # 'Final Energy|Residential and Commercial|Commercial|Solids|Coal', 

]


# Residential and commercial final energy (stacked area plot)
residential_commercial_final_energy = [
    # 'Final Energy|Residential and Commercial|Cooling', 
    # 'Final Energy|Residential and Commercial|Heating', 
    'Final Energy|Residential and Commercial|Electricity',
    'Final Energy|Residential and Commercial|Gases',
    # 'Final Energy|Residential and Commercial|Heat|Space', 
    'Final Energy|Residential and Commercial|Heat', 
    'Final Energy|Residential and Commercial|Liquids',
    'Final Energy|Residential and Commercial|Hydrogen', 
    'Final Energy|Residential and Commercial|Solids',
    # 'Final Energy|Residential and Commercial|Solids|Biomass',
    # 'Final Energy|Residential and Commercial|Solids|Biomass|Traditional', 
    # 'Final Energy|Residential and Commercial|Solids|Coal',
    # 'Final Energy|Residential and Commercial|Other', 

]

# # Electricity by sector (stacked area plot)
# residential_commercial_final_energy = [
#     'Final Energy|Residential and Commercial|Electricity',
#     'Final Energy|Transportation|ELectricity',
#     'Final Energy|Industry|Electricity', 
# ]


# Final energy transport by mode (stacked area plot)
transport_mode_final_energy = [
    # 'Final Energy|Transportation|Domestic Aviation', 
    # 'Final Energy|Transportation|Domestic Shipping', 
    # 'Final Energy|Transportation|Bus', 
    # 'Final Energy|Transportation|Light-Duty Vehicle', 
    # 'Final Energy|Transportation|Truck', 

    # 'Final Energy|Transportation|Aviation',
    # 'Final Energy|Transportation|Aviation|Passenger',
    # 'Final Energy|Transportation|Freight',
    # 'Final Energy|Transportation|Freight|Electricity',
    # 'Final Energy|Transportation|Freight|Hydrogen',
    # 'Final Energy|Transportation|Freight|Liquids',
    # 'Final Energy|Transportation|Freight|Other',
    # 'Final Energy|Transportation|Maritime',
    # 'Final Energy|Transportation|Maritime|Freight',
    # 'Final Energy|Transportation|Other',
    # 'Final Energy|Transportation|Passenger',
    # 'Final Energy|Transportation|Passenger|Electricity',
    # 'Final Energy|Transportation|Passenger|Gases',
    # 'Final Energy|Transportation|Passenger|Hydrogen',
    # 'Final Energy|Transportation|Passenger|Liquids',
    # 'Final Energy|Transportation|Rail',
    # 'Final Energy|Transportation|Rail|Freight',
    # 'Final Energy|Transportation|Rail|Passenger',
    # 'Final Energy|Transportation|Road',
    # 'Final Energy|Transportation|Road|Freight',
    # 'Final Energy|Transportation|Road|Passenger',
    # 'Final Energy|Transportation|Road|Passenger|2W&3W',
    # 'Final Energy|Transportation|Road|Passenger|4W',
    # 'Final Energy|Transportation|Road|Passenger|Bus'
    
]

# Final energy transport by carrier (stacked area plot)
transport_carrier_final_energy = [
    'Final Energy|Transportation|Electricity',
    'Final Energy|Transportation|Gases',
    'Final Energy|Transportation|Hydrogen',
    'Final Energy|Transportation|Liquids',
    'Final Energy|Transportation|Solids', 
    # 'Final Energy|Transportation|Other', 
    # 'Final Energy|Transportation|Liquids|Biomass', 
    # 'Final Energy|Transportation|Liquids|Oil', 



]



# Final energy industry by carrier (stacked area plot)
industry_carrier_final_energy = [
    'Final Energy|Industry|Electricity',
    'Final Energy|Industry|Gases',
    'Final Energy|Industry|Heat',
    'Final Energy|Industry|Hydrogen',
    'Final Energy|Industry|Liquids',
    'Final Energy|Industry|Other', 
    'Final Energy|Industry|Solids',
    # 'Final Energy|Industry|Solids|Biomass',
    # 'Final Energy|Industry|Solids|Coal',
]

# Final energy INDUSTRY by subsecto (stacked area plot)
industry_subsector_final_energy = [
    # 'Final Energy|Industry|Cement',
    # 'Final Energy|Industry|Cement|Electricity',
    # 'Final Energy|Industry|Cement|Gases',
    # 'Final Energy|Industry|Chemicals',
    # 'Final Energy|Industry|Chemicals|Ammonia',
    # 'Final Energy|Industry|Chemicals|Ammonia|Gases',
    # 'Final Energy|Industry|Chemicals|Ammonia|Hydrogen',
    # 'Final Energy|Industry|Chemicals|Ammonia|Liquids',
    # 'Final Energy|Industry|Chemicals|Ammonia|Solids',
    # 'Final Energy|Industry|Chemicals|Ammonia|Solids|Fossil',
    # 'Final Energy|Industry|Chemicals|Electricity',
    # 'Final Energy|Industry|Chemicals|Gases',
    # 'Final Energy|Industry|Chemicals|Heat',
    # 'Final Energy|Industry|Chemicals|Hydrogen',
    # 'Final Energy|Industry|Chemicals|Liquids',
    # 'Final Energy|Industry|Chemicals|Solids',
    # 'Final Energy|Industry|Chemicals|Solids|Bioenergy',
    # 'Final Energy|Industry|Chemicals|Solids|Fossil',
    # 'Final Energy|Industry|Non-ferrous metals',
    # 'Final Energy|Industry|Non-ferrous metals|Electricity',
    # 'Final Energy|Industry|Non-ferrous metals|Gases',
    # 'Final Energy|Industry|Non-ferrous metals|Liquids',
    # 'Final Energy|Industry|Non-ferrous metals|Solids',
    # 'Final Energy|Industry|Non-ferrous metals|Solids|Bioenergy',
    # 'Final Energy|Industry|Non-ferrous metals|Solids|Fossil',
    # 'Final Energy|Industry|Steel',
    # 'Final Energy|Industry|Steel|Electricity',
    # 'Final Energy|Industry|Steel|Hydrogen',
    # 'Final Energy|Industry|Steel|Liquids',
    # 'Final Energy|Industry|Steel|Solids',
    # 'Final Energy|Industry|Steel|Solids|Bioenergy',
    # 'Final Energy|Industry|Steel|Solids|Fossil'

]

# CO2 Emissions from transport (stacked area plot)
CO2_emissions_transport = [
    # 'Emissions|CO2|Energy|Demand|Transportation|Freight',
    # 'Emissions|CO2|Energy|Demand|Transportation|Passenger',
    # 'Emissions|CO2|Energy|Demand|Transportation',
]

In [52]:

capacity_additions
overall_capacity_electricity 
emissions 
CO2_emissions 
carbon_sequestration 
land_use_sequestration 
carbon_dioxide_removal 
timeseries_variables 
energy_investment 
primary_energy 
final_energy_by_carriers 
final_energy_by_sources 
finals_by_sector 
finals_by_sector_electricity 
secondary_energy_electricity 
secondary_energy_non_electric 
residential_final_energy 
commercial_final_energy 
residential_commercial_final_energy 
transport_mode_final_energy
transport_carrier_final_energy 
industry_carrier_final_energy 
industry_subsector_final_energy 
CO2_emissions_transport 

[]

In [53]:
### Updated Variable Group Naming Dictionaries ###

# Capacity Additions (all commented in original list)
capacity_additions_names = {
    # 'Capacity Additions|Electricity|Biomass': 'CapAdd|Elec|Biomass',
    # 'Capacity Additions|Electricity|Fossil': 'CapAdd|Elec|Fossil',
    # 'Capacity Additions|Electricity|Coal': 'CapAdd|Elec|Coal',
    # 'Capacity Additions|Electricity|Gas': 'CapAdd|Elec|Gas',
    # 'Capacity Additions|Electricity|Oil': 'CapAdd|Elec|Oil',
    # 'Capacity Additions|Electricity|Biomass|w/ CCS': 'CapAdd|Elec|Biomass|w/ CCS',
    # 'Capacity Additions|Electricity|Biomass|w/o CCS': 'CapAdd|Elec|Biomass|w/o CCS',
    # 'Capacity Additions|Electricity|Coal|w/ CCS': 'CapAdd|Elec|Coal|w/ CCS',
    # 'Capacity Additions|Electricity|Coal|w/o CCS': 'CapAdd|Elec|Coal|w/o CCS',
    # 'Capacity Additions|Electricity|Gas|w/ CCS': 'CapAdd|Elec|Gas|w/ CCS',
    # 'Capacity Additions|Electricity|Gas|w/o CCS': 'CapAdd|Elec|Gas|w/o CCS',
    # 'Capacity Additions|Electricity|Oil|w/ CCS': 'CapAdd|Elec|Oil|w/ CCS',
    # 'Capacity Additions|Electricity|Oil|w/o CCS': 'CapAdd|Elec|Oil|w/o CCS',
    # 'Capacity Additions|Electricity|Solar': 'CapAdd|Elec|Solar',
    # 'Capacity Additions|Electricity|Wind': 'CapAdd|Elec|Wind',
    # 'Capacity Additions|Electricity|Geothermal': 'CapAdd|Elec|Geothermal',
    # 'Capacity Additions|Electricity|Nuclear': 'CapAdd|Elec|Nuclear',
    # 'Capacity Additions|Electricity|Hydro': 'CapAdd|Elec|Hydro',
}

# Overall Capacity
overall_capacity_names = {
    'Capacity|Electricity|Biomass|w/ CCS': 'Cap|Elec|Biomass|w/ CCS',
    'Capacity|Electricity|Biomass|w/o CCS': 'Cap|Elec|Biomass|w/o CCS',
    'Capacity|Electricity|Biomass': 'Cap|Elec|Biomass',
    'Capacity|Electricity|Fossil': 'Cap|Elec|Fossil',
    'Capacity|Electricity|Coal': 'Cap|Elec|Coal',
    'Capacity|Electricity|Gas': 'Cap|Elec|Gas',
    'Capacity|Electricity|Coal|w/ CCS': 'Cap|Elec|Coal|w/ CCS',  
    'Capacity|Electricity|Coal|w/o CCS': 'Cap|Elec|Coal|w/o CCS',
    'Capacity|Electricity|Gas|w/ CCS': 'Cap|Elec|Gas|w/ CCS',
    'Capacity|Electricity|Gas|w/o CCS': 'Cap|Elec|Gas|w/o CCS',
    'Capacity|Electricity|Oil': 'Cap|Elec|Oil',
    'Capacity|Electricity|Solar': 'Cap|Elec|Solar',
    'Capacity|Electricity|Wind': 'Cap|Elec|Wind',
    'Capacity|Electricity|Geothermal': 'Cap|Elec|Geothermal',
    'Capacity|Electricity|Hydro': 'Cap|Elec|Hydro',
    'Capacity|Electricity|Nuclear': 'Cap|Elec|Nuclear',
    # 'Capacity|Electricity|Ocean': 'Cap|Elec|Ocean',
    # 'Capacity|Electricity|Solar|CSP': 'Cap|Elec|Solar|CSP',
    # 'Capacity|Electricity|Solar|PV': 'Cap|Elec|Solar|PV',
    # 'Capacity|Electricity|Wind|Offshore': 'Cap|Elec|Wind|Offshore',
    # 'Capacity|Electricity|Wind|Onshore': 'Cap|Elec|Wind|Onshore',
}

# Emissions (all commented in original list)
emissions_names = {
    # 'Emissions|CO2': 'CO2',
    # 'Emissions|CH4': 'CH4',
    # 'Emissions|Kyoto Gases': 'Kyoto Gases',
    # 'Emissions|N2O': 'N2O',
    # 'Emissions|NOx': 'NOx',
    # 'Emissions|BC': 'BC',
    # 'Emissions|CF4': 'CF4',
    # 'Emissions|C2F6': 'C2F6',
    # 'Emissions|CO': 'CO',
    # 'Emissions|F-Gases': 'F-Gases',
    # 'Emissions|HFC': 'HFC',
    # 'Emissions|NH3': 'NH3',
    # 'Emissions|OC': 'OC',
    # 'Emissions|SF6': 'SF6',
    # 'Emissions|Sulfur': 'Sulfur',
    # 'Emissions|VOC': 'VOC',
}

# CO2 Emissions (all commented in original list)
CO2_emissions_names = {
    # 'Emissions|CO2|AFOLU': 'CO2|AFOLU',
    # 'Emissions|CO2|Energy': 'CO2|Energy',
    # 'Emissions|CO2|Energy and Industrial Processes': 'CO2|Energy and IndProc',
    # 'Emissions|CO2|Fossil Fuels and Industry': 'CO2|Fossil Fuels and Industry',
    # 'Emissions|CO2|Industrial Processes': 'CO2|IndProc',
    # 'Emissions|CO2|Product Use': 'CO2|Product Use',
    # 'Emissions|CO2|Waste': 'CO2|Waste',
    # 'Emissions|CO2|Capture and Removal': 'CO2|Capture and Removal',
}

# Carbon Sequestration
carbon_sequestration_names = {
    'Carbon Sequestration|Direct Air Capture': 'DACCS',
    'Carbon Sequestration|CCS|Biomass': 'CSeq|CCS|Biomass',
    'Carbon Sequestration|CCS|Fossil': 'CSeq|CCS|Fossil',
    'Carbon Sequestration|CCS|Fossil|Energy|Demand|Industry': 'CSeq|CCS|Fossil|Industry',
    'Carbon Sequestration|CCS|Fossil|Energy|Supply': 'CSeq|CCS|Fossil|Supply',
    # 'Carbon Sequestration|CCS': 'CSeq|CCS',
    # 'Carbon Sequestration|CCS|Biomass|Energy|Supply': 'CSeq|CCS|Biomass|Supply',
    # 'Carbon Sequestration|CCS|Biomass|Energy|Supply|Electricity': 'CSeq|CCS|Biomass|Supply|Elec',
    # 'Carbon Sequestration|CCS|Biomass|Energy|Supply|Liquids': 'CSeq|CCS|Biomass|Supply|Liquids',
    # 'Carbon Sequestration|CCS|Biomass|Energy|Supply|Hydrogen': 'CSeq|CCS|Biomass|Supply|H2',
    # 'Carbon Sequestration|CCS|Biomass|Energy|Demand|Industry': 'CSeq|CCS|Biomass|Industry',
    # 'Carbon Sequestration|CCS|Fossil|Energy|Supply|Electricity': 'CSeq|CCS|Fossil|Supply|Elec',
    # 'Carbon Sequestration|CCS|Fossil|Energy|Supply|Hydrogen': 'CSeq|CCS|Fossil|Supply|H2',
    # 'Carbon Sequestration|CCS|Fossil|Energy|Supply|Liquids': 'CSeq|CCS|Fossil|Supply|Liquids',
    # 'Carbon Sequestration|CCS|Industrial Processes': 'CSeq|CCS|IndProc',
    # 'Carbon Sequestration|CCS|Direct Air Capture': 'CSeq|CCS|DAC',
    # 'Carbon Sequestration|Land Use': 'CSeq|Land Use',
    # 'Carbon Sequestration|CCS|Industry': 'CSeq|CCS|Industry',
}

# Land use sequestration (all commented in original list)
land_use_sequestration_names = {
    # 'Carbon Sequestration|Land Use|Afforestation': 'CSeq|Land Use|Afforestation',
    # 'Carbon Sequestration|Land Use|Agriculture': 'CSeq|Land Use|Agriculture',
    # 'Carbon Sequestration|Land Use|Forest Management': 'CSeq|Land Use|Forest Management',
    # 'Carbon Sequestration|Land Use|Other': 'CSeq|Land Use|Other',
    # 'Carbon Sequestration|Land Use|Other LUC': 'CSeq|Land Use|Other LUC',
}

# Carbon dioxide removal (all commented in original list)
carbon_dioxide_removal_names = {
    # 'Carbon Sequestration|Direct Air Capture': 'CDR|DAC',
    # 'Carbon Sequestration|Ocean': 'CDR|Ocean',
    # 'Carbon Sequestration|Soil Carbon': 'CDR|Soil Carbon',
    # 'Carbon Sequestration|Land Use|Afforestation': 'CDR|Land Use|Afforestation',
    # 'Carbon Sequestration|Land Use|Agriculture': 'CDR|Land Use|Agriculture',
    # 'Carbon Sequestration|Land Use|Forest Management': 'CDR|Land Use|Forest Management',
    # 'Carbon Sequestration|Land Use|Other': 'CDR|Land Use|Other',
    # 'Carbon Sequestration|Land Use|Other LUC': 'CDR|Land Use|Other LUC',
    # 'Carbon Removal': 'CDR',
    # 'Carbon Removal|Geological Storage|Direct Air Capture': 'CDR|Geol|DAC',
    # 'Carbon Removal|Geological Storage': 'CDR|Geol',
    # 'Carbon Removal|Geological Storage|Biomass': 'CDR|Geol|Biomass',
    # 'Carbon Removal|Land Use': 'CDR|Land Use',
    # 'Carbon Capture|Geological Storage|Direct Air Capture': 'CDR|Geol|DAC',
    # 'Carbon Capture|Geological Storage|Other Sources': 'CDR|Geol|Other',
    # 'Carbon Capture|Industrial Processes': 'CDR|IndProc',
    # 'Gross Removals|CO2': 'Gross Removals|CO2',
}

# Overall picture (Timeseries Variables)
timeseries_variables_names = {
    'Carbon Sequestration|CCS': 'CSeq|CCS',
    'Final Energy': 'Final Energy',
    'Capacity|Electricity': 'Cap|Elec',
    'CO2|Energy and Industrial Processes': 'CO2|Energy and IndProc',
    'Emissions|CO2|Energy and Industrial Processes': 'CO2|Energy and IndProc',
    'Final Energy|Electricity': 'FE|Electricity',
    'Secondary Energy|Electricity': 'SE|Electricity',
    
    # 'Carbon Sequestration|Land Use': 'CSeq|Land Use',
    # 'Primary Energy': 'Primary Energy',
    # 'GDP|PPP': 'GDP|PPP',
    # 'Emissions|CO2': 'CO2 Emissions',
    # 'Temperature|Global Mean': 'Temperature|Global Mean',
    # 'Investment|Energy Supply|Electricity': 'Inv|Elec',
    # 'Land Cover': 'Land Cover',
    # 'Secondary Energy': 'Secondary Energy',
    # 'Capacity Additions|Electricity': 'CapAdd|Elec',
    # 'Price|Carbon': 'Price|Carbon',
    # 'Price|Fossil': 'Price|Fossil',
    # 'Price|Electricity': 'Price|Electricity',

}

# Investment in energy supply (all commented in original list)
energy_investment_names = {
    # 'Investment|Energy Supply|CO2 Transport and Storage': 'Inv|CO2 T&S',
    # 'Investment|Energy Supply|Electricity': 'Inv|Elec',
    # 'Investment|Energy Supply|Heat': 'Inv|Heat',
    # 'Investment|Energy Supply|Hydrogen': 'Inv|H2',
    # 'Investment|Energy Supply|Liquids': 'Inv|Liquids',
    # 'Investment|Energy Supply|Other': 'Inv|Other',
    # 'Investment|Energy Supply|Electricity|Biomass|w/ CCS': 'Inv|Elec|Biomass|w/ CCS',
    # 'Investment|Energy Supply|Electricity|Biomass|w/o CCS': 'Inv|Elec|Biomass|w/o CCS',
    # 'Investment|Energy Supply|Electricity|Coal|w/ CCS': 'Inv|Elec|Coal|w/ CCS',
    # 'Investment|Energy Supply|Electricity|Coal|w/o CCS': 'Inv|Elec|Coal|w/o CCS',
    # 'Investment|Energy Supply|Electricity|Gas|w/ CCS': 'Inv|Elec|Gas|w/ CCS',
    # 'Investment|Energy Supply|Electricity|Gas|w/o CCS': 'Inv|Elec|Gas|w/o CCS',
    # 'Investment|Energy Supply|Electricity|Oil|w/ CCS': 'Inv|Elec|Oil|w/ CCS',
    # 'Investment|Energy Supply|Electricity|Oil|w/o CCS': 'Inv|Elec|Oil|w/o CCS',
    # 'Investment|Energy Supply|Electricity|Solar': 'Inv|Elec|Solar',
    # 'Investment|Energy Supply|Electricity|Wind': 'Inv|Elec|Wind',
    # 'Investment|Energy Supply|Electricity|Geothermal': 'Inv|Elec|Geothermal',
    # 'Investment|Energy Supply|Electricity|Hydro': 'Inv|Elec|Hydro',
    # 'Investment|Energy Supply|Electricity|Nuclear': 'Inv|Elec|Nuclear',
    # 'Investment|Energy Supply|Electricity|Transmission and Distribution': 'Inv|Elec|T&D',
}

# Primary energy (all commented in original list)
primary_energy_names = {
    # 'Primary Energy|Biomass': 'PE|Biomass',
    # 'Primary Energy|Biomass|w/ CCS': 'PE|Biomass|w/ CCS',
    # 'Primary Energy|Biomass|w/o CCS': 'PE|Biomass|w/o CCS',
    # 'Primary Energy|Coal': 'PE|Coal',
    # 'Primary Energy|Coal|w/ CCS': 'PE|Coal|w/ CCS',
    # 'Primary Energy|Coal|w/o CCS': 'PE|Coal|w/o CCS',
    # 'Primary Energy|Gas': 'PE|Gas',
    # 'Primary Energy|Gas|w/ CCS': 'PE|Gas|w/ CCS',
    # 'Primary Energy|Gas|w/o CCS': 'PE|Gas|w/o CCS',
    # 'Primary Energy|Geothermal': 'PE|Geothermal',
    # 'Primary Energy|Hydro': 'PE|Hydro',
    # 'Primary Energy|Nuclear': 'PE|Nuclear',
    # 'Primary Energy|Ocean': 'PE|Ocean',
    # 'Primary Energy|Oil': 'PE|Oil',
    # 'Primary Energy|Oil|w/ CCS': 'PE|Oil|w/ CCS',
    # 'Primary Energy|Oil|w/o CCS': 'PE|Oil|w/o CCS',
    # 'Primary Energy|Other': 'PE|Other',
    # 'Primary Energy|Secondary Energy Trade': 'PE|Secondary Energy Trade',
    # 'Primary Energy|Solar': 'PE|Solar',
    # 'Primary Energy|Wind': 'PE|Wind',
    'Primary Energy|Non-Biomass Renewables': 'Non-Bio Renewables',
}

# Final energy by carriers
final_energy_by_carriers_names = {
    'Final Energy|Electricity': 'FE|Electricity',
    'Final Energy|Gases': 'FE|Gases',
    'Final Energy|Heat': 'FE|Heat',
    'Final Energy|Liquids': 'FE|Liquids',
    'Final Energy|Solids': 'FE|Solids',
    # 'Final Energy|Solids|Biomass': 'FE|Solids|Biomass',
    # 'Final Energy|Solids|Biomass|Traditional': 'FE|Solids|Biomass|Trad',
    # 'Final Energy|Solids|Coal': 'FE|Solids|Coal',
    # 'Final Energy|Other Sector': 'FE|Other Sector',
}

# Final energy by sources
final_energy_by_sources_names = {
    'Final Energy|Geothermal': 'FE|Geothermal',
    'Final Energy|Solar': 'FE|Solar',
    'Final Energy|Hydrogen': 'FE|Hydrogen',
}

# Finals by sector
finals_by_sector_names = {
    'Final Energy|Industry': 'FE|Industry',
    'Final Energy|Non-Energy Use': 'FE|Non-Energy Use',
    'Final Energy|Residential and Commercial': 'FE|Res & Com',
    'Final Energy|Transportation': 'FE|Transportation',
    # 'Final Energy|Agriculture': 'FE|Agriculture',
    # 'Final Energy|Bunkers': 'FE|Bunkers',
    # 'Final Energy|Commercial': 'FE|Commercial',
    # 'Final Energy|Residential': 'FE|Residential',
    # 'Final Energy|Transportation (w/ bunkers)': 'FE|Transport (w/ bunkers)',
    # 'Final Energy|Other Sector': 'FE|Other Sector',
    # 'Final Energy|Non-Energy Use|Coal': 'FE|Non-Energy Use|Coal',
    # 'Final Energy|Non-Energy Use|Gas': 'FE|Non-Energy Use|Gas',
    # 'Final Energy|Non-Energy Use|Oil': 'FE|Non-Energy Use|Oil',
    # 'Final Energy|Non-Energy Use|Biomass': 'FE|Non-Energy Use|Biomass',
}

# Final energy by sector for electricity
finals_by_sector_electricity_names = {
    'Final Energy|Industry|Electricity': 'FE|Ind|Electricity',
    'Final Energy|Residential and Commercial|Electricity': 'FE|ResCom|Electricity',
    'Final Energy|Transportation|Electricity': 'FE|Transport|Electricity',
}

# Secondary energy (Electricity)
secondary_energy_electricity_names = {
    'Secondary Energy|Electricity|Biomass': 'SE|Elec|Biomass',
    'Secondary Energy|Electricity|Coal': 'SE|Elec|Coal',
    'Secondary Energy|Electricity|Gas': 'SE|Elec|Gas',
    'Secondary Energy|Electricity|Oil': 'SE|Elec|Oil',
    'Secondary Energy|Electricity|Geothermal': 'SE|Elec|Geothermal',
    'Secondary Energy|Electricity|Hydro': 'SE|Elec|Hydro',
    'Secondary Energy|Electricity|Nuclear': 'SE|Elec|Nuclear',
    'Secondary Energy|Electricity|Solar': 'SE|Elec|Solar',
    'Secondary Energy|Electricity|Wind': 'SE|Elec|Wind',
    'Secondary Energy|Electricity|Ocean': 'SE|Elec|Ocean',
    # 'Secondary Energy|Electricity|Biomass|w/ CCS': 'SE|Elec|Biomass|w/ CCS',
    # 'Secondary Energy|Electricity|Biomass|w/o CCS': 'SE|Elec|Biomass|w/o CCS',
    # 'Secondary Energy|Electricity|Coal|w/ CCS': 'SE|Elec|Coal|w/ CCS',
    # 'Secondary Energy|Electricity|Coal|w/o CCS': 'SE|Elec|Coal|w/o CCS',
    # 'Secondary Energy|Electricity|Gas|w/ CCS': 'SE|Elec|Gas|w/ CCS',
    # 'Secondary Energy|Electricity|Gas|w/o CCS': 'SE|Elec|Gas|w/o CCS',
    # 'Secondary Energy|Electricity|Oil|w/o CCS': 'SE|Elec|Oil|w/o CCS',
    # 'Secondary Energy|Electricity|Non-Biomass Renewables': 'SE|Elec|Non-Bio Renewables',
    # 'Secondary Energy|Electricity|Solar|CSP': 'SE|Elec|Solar|CSP',
    # 'Secondary Energy|Electricity|Solar|PV': 'SE|Elec|Solar|PV',
    # 'Secondary Energy|Electricity|Wind|Offshore': 'SE|Elec|Wind|Offshore',
    # 'Secondary Energy|Electricity|Wind|Onshore': 'SE|Elec|Wind|Onshore',
}

# Secondary energy (Non-Electric)
secondary_energy_non_electric_names = {
    'Secondary Energy|Gases': 'SE|Gases',
    'Secondary Energy|Hydrogen': 'SE|Hydrogen',
    'Secondary Energy|Liquids': 'SE|Liquids',
    'Secondary Energy|Solids': 'SE|Solids',
    # 'Secondary Energy|Hydrogen|Electricity': 'SE|Hydrogen|Electricity',
    # 'Secondary Energy|Hydrogen|Fossil': 'SE|Hydrogen|Fossil',
    # 'Secondary Energy|Hydrogen|Fossil|w/ CCS': 'SE|Hydrogen|Fossil|w/ CCS',
    # 'Secondary Energy|Hydrogen|Fossil|w/o CCS': 'SE|Hydrogen|Fossil|w/o CCS',
    # 'Secondary Energy|Liquids|Biomass': 'SE|Liquids|Biomass',
    # 'Secondary Energy|Liquids|Biomass|w/ CCS': 'SE|Liquids|Biomass|w/ CCS',
    # 'Secondary Energy|Liquids|Biomass|w/o CCS': 'SE|Liquids|Biomass|w/o CCS',
    # 'Secondary Energy|Liquids|Gas': 'SE|Liquids|Gas',
    # 'Secondary Energy|Liquids|Gas|w/o CCS': 'SE|Liquids|Gas|w/o CCS',
    # 'Secondary Energy|Liquids|Oil': 'SE|Liquids|Oil',
}

# Residential final energy (all commented in original list)
residential_final_energy_names = {
    # 'Final Energy|Residential and Commercial|Residential|Cooling': 'FE|Res|Cooling',
    # 'Final Energy|Residential and Commercial|Residential|Electricity': 'FE|Res|Electricity',
    # 'Final Energy|Residential and Commercial|Residential|Gases': 'FE|Res|Gases',
    # 'Final Energy|Residential and Commercial|Residential|Heat': 'FE|Res|Heat',
    # 'Final Energy|Residential and Commercial|Residential|Heating|Space': 'FE|Res|Heating|Space',
    # 'Final Energy|Residential and Commercial|Residential|Hydrogen': 'FE|Res|Hydrogen',
    # 'Final Energy|Residential and Commercial|Residential|Liquids': 'FE|Res|Liquids',
}

# Commercial final energy (all commented in original list)
commercial_final_energy_names = {
    # 'Final Energy|Residential and Commercial|Commercial|Cooling': 'FE|Com|Cooling',
    # 'Final Energy|Residential and Commercial|Commercial|Electricity': 'FE|Com|Electricity',
    # 'Final Energy|Residential and Commercial|Commercial|Gases': 'FE|Com|Gases',
    # 'Final Energy|Residential and Commercial|Commercial|Heat': 'FE|Com|Heat',
    # 'Final Energy|Residential and Commercial|Commercial|Heating|Space': 'FE|Com|Heating|Space',
    # 'Final Energy|Residential and Commercial|Commercial|Hydrogen': 'FE|Com|Hydrogen',
    # 'Final Energy|Residential and Commercial|Commercial|Liquids': 'FE|Com|Liquids',
    # 'Final Energy|Residential and Commercial|Commercial|Solids': 'FE|Com|Solids',
    # 'Final Energy|Residential and Commercial|Commercial|Solids|Biomass': 'FE|Com|Solids|Biomass',
    # 'Final Energy|Residential and Commercial|Commercial|Solids|Coal': 'FE|Com|Solids|Coal',
}

# Residential and commercial final energy (combined)
residential_commercial_final_energy_names = {
    'Final Energy|Residential and Commercial|Electricity': 'FE|ResCom|Electricity',
    'Final Energy|Residential and Commercial|Gases': 'FE|ResCom|Gases',
    'Final Energy|Residential and Commercial|Heat': 'FE|ResCom|Heat',
    'Final Energy|Residential and Commercial|Liquids': 'FE|ResCom|Liquids',
    'Final Energy|Residential and Commercial|Hydrogen': 'FE|ResCom|Hydrogen',
    'Final Energy|Residential and Commercial|Solids': 'FE|ResCom|Solids',
    # 'Final Energy|Residential and Commercial|Cooling': 'FE|ResCom|Cooling',
    # 'Final Energy|Residential and Commercial|Heating': 'FE|ResCom|Heating',
    # 'Final Energy|Residential and Commercial|Heat|Space': 'FE|ResCom|Heat|Space',
    # 'Final Energy|Residential and Commercial|Solids|Biomass': 'FE|ResCom|Solids|Biomass',
    # 'Final Energy|Residential and Commercial|Solids|Biomass|Traditional': 'FE|ResCom|Solids|Biomass|Trad',
    # 'Final Energy|Residential and Commercial|Solids|Coal': 'FE|ResCom|Solids|Coal',
    # 'Final Energy|Residential and Commercial|Other': 'FE|ResCom|Other',
}

# Final energy transport by mode (all commented in original list)
transport_mode_final_energy_names = {
    # 'Final Energy|Transportation|Domestic Aviation': 'FE|Transport|Dom Aviation',
    # 'Final Energy|Transportation|Domestic Shipping': 'FE|Transport|Dom Shipping',
    # 'Final Energy|Transportation|Bus': 'FE|Transport|Bus',
    # 'Final Energy|Transportation|Light-Duty Vehicle': 'FE|Transport|LDV',
    # 'Final Energy|Transportation|Truck': 'FE|Transport|Truck',
    # 'Final Energy|Transportation|Aviation': 'FE|Transport|Aviation',
    # 'Final Energy|Transportation|Aviation|Passenger': 'FE|Transport|Aviation|Passenger',
    # 'Final Energy|Transportation|Freight': 'FE|Transport|Freight',
    # 'Final Energy|Transportation|Freight|Electricity': 'FE|Transport|Freight|Electricity',
    # 'Final Energy|Transportation|Freight|Hydrogen': 'FE|Transport|Freight|Hydrogen',
    # 'Final Energy|Transportation|Freight|Liquids': 'FE|Transport|Freight|Liquids',
    # 'Final Energy|Transportation|Freight|Other': 'FE|Transport|Freight|Other',
    # 'Final Energy|Transportation|Maritime': 'FE|Transport|Maritime',
    # 'Final Energy|Transportation|Maritime|Freight': 'FE|Transport|Maritime|Freight',
    # 'Final Energy|Transportation|Other': 'FE|Transport|Other',
    # 'Final Energy|Transportation|Passenger': 'FE|Transport|Passenger',
    # 'Final Energy|Transportation|Passenger|Electricity': 'FE|Transport|Passenger|Electricity',
    # 'Final Energy|Transportation|Passenger|Gases': 'FE|Transport|Passenger|Gases',
    # 'Final Energy|Transportation|Passenger|Hydrogen': 'FE|Transport|Passenger|Hydrogen',
    # 'Final Energy|Transportation|Passenger|Liquids': 'FE|Transport|Passenger|Liquids',
    # 'Final Energy|Transportation|Rail': 'FE|Transport|Rail',
    # 'Final Energy|Transportation|Rail|Freight': 'FE|Transport|Rail|Freight',
    # 'Final Energy|Transportation|Rail|Passenger': 'FE|Transport|Rail|Passenger',
    # 'Final Energy|Transportation|Road': 'FE|Transport|Road',
    # 'Final Energy|Transportation|Road|Freight': 'FE|Transport|Road|Freight',
    # 'Final Energy|Transportation|Road|Passenger': 'FE|Transport|Road|Passenger',
    # 'Final Energy|Transportation|Road|Passenger|2W&3W': 'FE|Transport|Road|Passenger|2W&3W',
    # 'Final Energy|Transportation|Road|Passenger|4W': 'FE|Transport|Road|Passenger|4W',
    # 'Final Energy|Transportation|Road|Passenger|Bus': 'FE|Transport|Road|Passenger|Bus',
}

# Final energy transport by carrier
transport_carrier_final_energy_names = {
    'Final Energy|Transportation|Electricity': 'FE|Transport|Electricity',
    'Final Energy|Transportation|Gases': 'FE|Transport|Gases',
    'Final Energy|Transportation|Hydrogen': 'FE|Transport|Hydrogen',
    'Final Energy|Transportation|Liquids': 'FE|Transport|Liquids',
    'Final Energy|Transportation|Solids': 'FE|Transport|Solids',
    # 'Final Energy|Transportation|Other': 'FE|Transport|Other',
    # 'Final Energy|Transportation|Liquids|Biomass': 'FE|Transport|Liquids|Biomass',
    # 'Final Energy|Transportation|Liquids|Oil': 'FE|Transport|Liquids|Oil',
}

# Final energy industry by carrier
industry_carrier_final_energy_names = {
    'Final Energy|Industry|Electricity': 'FE|Ind|Electricity',
    'Final Energy|Industry|Gases': 'FE|Ind|Gases',
    'Final Energy|Industry|Heat': 'FE|Ind|Heat',
    'Final Energy|Industry|Hydrogen': 'FE|Ind|Hydrogen',
    'Final Energy|Industry|Liquids': 'FE|Ind|Liquids',
    'Final Energy|Industry|Other': 'FE|Ind|Other',
    'Final Energy|Industry|Solids': 'FE|Ind|Solids',
    # 'Final Energy|Industry|Solids|Biomass': 'FE|Ind|Solids|Biomass',
    # 'Final Energy|Industry|Solids|Coal': 'FE|Ind|Solids|Coal',
}

# Final energy industry by subsector (all commented in original list)
industry_subsector_final_energy_names = {
    # 'Final Energy|Industry|Cement': 'FE|Ind|Cement',
    # 'Final Energy|Industry|Cement|Electricity': 'FE|Ind|Cement|Electricity',
    # 'Final Energy|Industry|Cement|Gases': 'FE|Ind|Cement|Gases',
    # 'Final Energy|Industry|Chemicals': 'FE|Ind|Chemicals',
    # 'Final Energy|Industry|Chemicals|Ammonia': 'FE|Ind|Chemicals|Ammonia',
    # 'Final Energy|Industry|Chemicals|Ammonia|Gases': 'FE|Ind|Chemicals|Ammonia|Gases',
    # 'Final Energy|Industry|Chemicals|Ammonia|Hydrogen': 'FE|Ind|Chemicals|Ammonia|Hydrogen',
    # 'Final Energy|Industry|Chemicals|Ammonia|Liquids': 'FE|Ind|Chemicals|Ammonia|Liquids',
    # 'Final Energy|Industry|Chemicals|Ammonia|Solids': 'FE|Ind|Chemicals|Ammonia|Solids',
    # 'Final Energy|Industry|Chemicals|Ammonia|Solids|Fossil': 'FE|Ind|Chemicals|Ammonia|Solids|Fossil',
    # 'Final Energy|Industry|Chemicals|Electricity': 'FE|Ind|Chemicals|Electricity',
    # 'Final Energy|Industry|Chemicals|Gases': 'FE|Ind|Chemicals|Gases',
    # 'Final Energy|Industry|Chemicals|Heat': 'FE|Ind|Chemicals|Heat',
    # 'Final Energy|Industry|Chemicals|Hydrogen': 'FE|Ind|Chemicals|Hydrogen',
    # 'Final Energy|Industry|Chemicals|Liquids': 'FE|Ind|Chemicals|Liquids',
    # 'Final Energy|Industry|Chemicals|Solids': 'FE|Ind|Chemicals|Solids',
    # 'Final Energy|Industry|Chemicals|Solids|Bioenergy': 'FE|Ind|Chemicals|Solids|Bioenergy',
    # 'Final Energy|Industry|Chemicals|Solids|Fossil': 'FE|Ind|Chemicals|Solids|Fossil',
    # 'Final Energy|Industry|Non-ferrous metals': 'FE|Ind|Non-ferrous metals',
    # 'Final Energy|Industry|Non-ferrous metals|Electricity': 'FE|Ind|Non-ferrous metals|Electricity',
    # 'Final Energy|Industry|Non-ferrous metals|Gases': 'FE|Ind|Non-ferrous metals|Gases',
    # 'Final Energy|Industry|Non-ferrous metals|Liquids': 'FE|Ind|Non-ferrous metals|Liquids',
    # 'Final Energy|Industry|Non-ferrous metals|Solids': 'FE|Ind|Non-ferrous metals|Solids',
    # 'Final Energy|Industry|Non-ferrous metals|Solids|Bioenergy': 'FE|Ind|Non-ferrous metals|Solids|Bioenergy',
    # 'Final Energy|Industry|Non-ferrous metals|Solids|Fossil': 'FE|Ind|Non-ferrous metals|Solids|Fossil',
    # 'Final Energy|Industry|Steel': 'FE|Ind|Steel',
    # 'Final Energy|Industry|Steel|Electricity': 'FE|Ind|Steel|Electricity',
    # 'Final Energy|Industry|Steel|Hydrogen': 'FE|Ind|Steel|Hydrogen',
    # 'Final Energy|Industry|Steel|Liquids': 'FE|Ind|Steel|Liquids',
    # 'Final Energy|Industry|Steel|Solids': 'FE|Ind|Steel|Solids',
    # 'Final Energy|Industry|Steel|Solids|Bioenergy': 'FE|Ind|Steel|Solids|Bioenergy',
    # 'Final Energy|Industry|Steel|Solids|Fossil': 'FE|Ind|Steel|Solids|Fossil',
}

# CO2 Emissions from transport (all commented in original list)
CO2_emissions_transport_names = {
    # 'Emissions|CO2|Energy|Demand|Transportation|Freight': 'CO2|Transport|Freight',
    # 'Emissions|CO2|Energy|Demand|Transportation|Passenger': 'CO2|Transport|Passenger',
    # 'Emissions|CO2|Energy|Demand|Transportation': 'CO2|Transport',
}

# --- Combine all renaming dictionaries ---
all_vars_names = {}
all_vars_names.update(capacity_additions_names)
all_vars_names.update(overall_capacity_names)
all_vars_names.update(emissions_names)
all_vars_names.update(CO2_emissions_names)
all_vars_names.update(carbon_sequestration_names)
all_vars_names.update(land_use_sequestration_names)
all_vars_names.update(carbon_dioxide_removal_names)
all_vars_names.update(timeseries_variables_names)
all_vars_names.update(energy_investment_names)
all_vars_names.update(primary_energy_names)
all_vars_names.update(final_energy_by_carriers_names)
all_vars_names.update(final_energy_by_sources_names)
all_vars_names.update(finals_by_sector_names)
all_vars_names.update(finals_by_sector_electricity_names)
all_vars_names.update(secondary_energy_electricity_names)
all_vars_names.update(secondary_energy_non_electric_names)
all_vars_names.update(residential_final_energy_names)
all_vars_names.update(commercial_final_energy_names)
all_vars_names.update(residential_commercial_final_energy_names)
all_vars_names.update(transport_mode_final_energy_names)
all_vars_names.update(transport_carrier_final_energy_names)
all_vars_names.update(industry_carrier_final_energy_names)
all_vars_names.update(industry_subsector_final_energy_names)
all_vars_names.update(CO2_emissions_transport_names)

# 4) Plotting Functions

In [54]:
# --------------------------
# Set up the style parameters
# --------------------------
sns.set()
sns.set_style("white")
sns.set_context("talk")
# sns.set_palette("colorblind")
plt.rcParams['figure.dpi']= 100
plt.rc("savefig", dpi=150)
plt.rc("font", size=14)
# Name of the scenarios on plots
scenario_names = {
    # 'NDC_LTT': 'D2_LTT',
    'NDC_LTT_DERP2_DACCS_2_4': 'D2_LTT',
    # 'NDC_LTT_DERP1_DACCS_3_6': 'D2_LTT_DACCS_3_6',
    # 'DERP3_MR_HP': 'D3_MR_HP',
    'DERP3_MR_HP_PriceShock': 'D3_MR_HP',
    # 'DERP3_MR_NG': 'D3_MR_NG',
    'DERP3_MR_NG_PriceShock': 'D3_MR_NG',
    # 'DERP5_SR_NG_HP': 'D5_SR_NG_HP',
    'DERP5_SR_NG_HP_PriceShock': 'D5_SR_NG_HP',    
}

# Dictionary for the colors based on scenario
colours = {
    'D2_NDC_DACCS_2_4':        '#000000',  # black
    'D3_HP_PriceShock':         '#FF7F00',  # orange
    'D3_NG_PriceShock':         '#1F78B4',  # blue
    'D5_NG_HP_PriceShock':      '#33A02C',  # dark green
}

# Dictionary for line styles based on model
model_linestyles = {
    'GCAM 7.0': '-',           # solid
    'TIAM_Grantham': '--',     # dashed

}

# Define a safe function to sanitise the variable name for saving files
def sanitize_filename(filename):
    return filename.replace("|", "_").replace("/", "_").replace("*", "")


In [55]:
scenarios_of_interest = [
   'D2_NDC_DACCS_2_4',
    'D3_HP_PriceShock',
    'D3_NG_PriceShock',
    'D5_NG_HP_PriceShock',
]

models_of_interest = [
    'GCAM 7.0',
    'TIAM_Grantham'
]


 # 4) Figure 3

In [56]:
groups = {
    # 'secondary_energy_non_electric': secondary_energy_non_electric,

    'Installed Electricity Capacity': overall_capacity_electricity,
    'Carbon Capture and Storage': carbon_sequestration,
    'Final Energy by Carriers': final_energy_by_carriers,
    'Final Energy by Sources': final_energy_by_sources,
    'Final Energy by Sector': finals_by_sector,
    'Electricity Use by Sector': finals_by_sector_electricity,
    'Electricity Generation by Source': secondary_energy_electricity,
    'Buildings Energy Demand': residential_commercial_final_energy,
    'Transport Energy by Carrier': transport_carrier_final_energy,
    'Industry Energy by Carrier': industry_carrier_final_energy,
    # primary energy
    'Primary Energy by Source': primary_energy,
    
}



In [57]:
# For all models calculate the share of Final Energy|Electricity of total Final Energy
# Calculate electricity share of total final energy
electricity_var = 'Final Energy|Electricity'
total_var = 'Final Energy'

# Extract data for both variables
electricity_df = df.filter(variable=electricity_var).data
total_df = df.filter(variable=total_var).data

# Rename value columns for clarity
electricity_df = electricity_df.rename(columns={'value': 'electricity_value'})
total_df = total_df.rename(columns={'value': 'total_value'})

# Merge the dataframes
merged = pd.merge(
    electricity_df[['model', 'scenario', 'region', 'year', 'electricity_value']],
    total_df[['model', 'scenario', 'region', 'year', 'total_value']],
    on=['model', 'scenario', 'region', 'year'],
    how='inner'
)

# Calculate the share
merged['electricity_share'] = (merged['electricity_value'] / merged['total_value']) * 100

# Handle any division by zero or invalid values
merged['electricity_share'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Quick look at the results
print("Electricity share summary statistics by model:")
print(merged.groupby('model')['electricity_share'].describe())

# Check for any missing or problematic values
print("\nMissing electricity share values by model:")
print(merged[merged['electricity_share'].isna()].groupby('model').size())

Electricity share summary statistics by model:
                count       mean        std       min        25%        50%  \
model                                                                         
GCAM 7.0       2640.0  33.702555  14.643719  1.697111  22.186852  33.565384   
TIAM_Grantham   704.0  35.803528  15.454066  7.211114  24.711377  33.011694   

                     75%        max  
model                                
GCAM 7.0       43.512498  75.089936  
TIAM_Grantham  47.214999  71.288092  

Missing electricity share values by model:
Series([], dtype: int64)


In [58]:
# First, prepare your data with baseline comparisons
baseline_scenario = 'D2_NDC_DACCS_2_4'

# Create the df_with_changes DataFrame 
if 'baseline_value' not in df.data.columns:
    # Create a DataFrame with baseline values
    baseline_df = df.data[df.data['scenario'] == baseline_scenario].copy()
    baseline_df = baseline_df.rename(columns={'value': 'baseline_value'})
    baseline_df = baseline_df[['model', 'region', 'variable', 'year', 'baseline_value']]
    
    # Merge with original data and calculate changes
    merged_df = pd.merge(df.data, baseline_df, on=['model', 'region', 'variable', 'year'], how='left')
    merged_df['delta'] = merged_df['value'] - merged_df['baseline_value']
    merged_df['percentage_change'] = (merged_df['delta'] / merged_df['baseline_value']) * 100
    
    # Replace infinite values (from division by zero) with NaN
    merged_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Store the enhanced DataFrame
    df_with_changes = merged_df
else:
    df_with_changes = df.data

In [59]:
df_with_changes

,model,scenario,region,variable,unit,year,value,baseline_value,delta,percentage_change
0,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Agricultural Demand,million t DM/yr,2005,150.331528,150.331528,0.000000,0.000000
1,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Agricultural Demand,million t DM/yr,2010,169.903455,169.903455,0.000000,0.000000
2,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Agricultural Demand,million t DM/yr,2015,197.688237,197.688237,0.000000,0.000000
3,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Agricultural Demand,million t DM/yr,2020,230.204863,230.204863,0.000000,0.000000
4,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Agricultural Demand,million t DM/yr,2025,265.096962,265.096962,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1314299,TIAM_Grantham,D5_NG_HP_PriceShock,World,Secondary Energy|Solids,EJ/yr,2060,60.597757,58.403481,2.194276,3.757098
1314300,TIAM_Grantham,D5_NG_HP_PriceShock,World,Secondary Energy|Solids,EJ/yr,2070,73.100953,71.651999,1.448955,2.022211
1314301,TIAM_Grantham,D5_NG_HP_PriceShock,World,Secondary Energy|Solids,EJ/yr,2080,82.142185,79.576822,2.565362,3.223756
1314302,TIAM_Grantham,D5_NG_HP_PriceShock,World,Secondary Energy|Solids,EJ/yr,2090,88.760887,86.070970,2.689916,3.125231


In [60]:
df_with_changes.query("variable == 'Energy System Cost'")

,model,scenario,region,variable,unit,year,value,baseline_value,delta,percentage_change
4100,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Energy System Cost,M$(2010)/yr,2005,57662.0,57662.0,0.0,0.000000
4101,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Energy System Cost,M$(2010)/yr,2010,87532.0,87532.0,0.0,0.000000
4102,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Energy System Cost,M$(2010)/yr,2020,161544.0,161544.0,0.0,0.000000
4103,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Energy System Cost,M$(2010)/yr,2030,254414.0,254414.0,0.0,0.000000
4104,GCAM 7.0,D2_NDC_DACCS_2_4,Africa_Eastern,Energy System Cost,M$(2010)/yr,2040,396844.0,396844.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
1313309,TIAM_Grantham,D5_NG_HP_PriceShock,World,Energy System Cost,M$(2010)/yr,2060,58526246.0,59508111.0,-981865.0,-1.649968
1313310,TIAM_Grantham,D5_NG_HP_PriceShock,World,Energy System Cost,M$(2010)/yr,2070,61018050.0,61403165.0,-385115.0,-0.627191
1313311,TIAM_Grantham,D5_NG_HP_PriceShock,World,Energy System Cost,M$(2010)/yr,2080,62806070.0,62902127.0,-96057.0,-0.152709
1313312,TIAM_Grantham,D5_NG_HP_PriceShock,World,Energy System Cost,M$(2010)/yr,2090,65230618.0,65290868.0,-60250.0,-0.092279


In [61]:
# ----------------------------------------------------------------
#  DACCS dashboard v4 - Modified layout
# ----------------------------------------------------------------

# ----------------------------------------------------------------
# Style helpers
# ----------------------------------------------------------------
model_linestyles = {"GCAM 7.0": "-", "TIAM_Grantham": "--"}

def _display_name(var: str, mapping: Dict[str, str]) -> str:
    return mapping.get(var, var.split("|")[-1])

def _model_markers(models: List[str]) -> Dict[str, str]:
    """Assign markers to models"""
    base = ["s", "s", "D", "v", "<", ">"]
    return {m: ("P" if m == "GCAM 7.0" else base[i % len(base)])
            for i, m in enumerate(models)}

# ----------------------------------------------------------------
# DACCS timeseries - spans 2 columns
# ----------------------------------------------------------------
def _ax_timeseries_daccs(df, *, ax, region, models, scenarios,
                         all_vars, colours):
    var = "Carbon Sequestration|Direct Air Capture"
    sub = (df.filter(variable=var, region=region)
             .filter(model=models).filter(scenario=scenarios))
    if sub.empty:
        ax.text(.5, .5, "No data", ha="center", va="center", fontsize=12)
        ax.axis("off");  return

    unit = sub.data["unit"].iloc[0]
    for m in models:
        ls = model_linestyles.get(m, "-")
        for s in scenarios:
            seri = sub.filter(model=m, scenario=s)
            if seri.empty:  continue
            ts = (seri.timeseries().reset_index()
                        .melt(id_vars=["model","scenario","region",
                                       "variable","unit"],
                              var_name="year", value_name="value"))
            ts["year"] = pd.to_numeric(ts["year"], errors="coerce")
            ax.plot(ts["year"], ts["value"],
                    color=colours.get(s,"black"), linestyle=ls, lw=2.5)

    ax.set_xlim(2025, 2100)
    ax.set_xlabel("Year", fontsize=11, fontweight='bold')
    ax.set_ylabel(unit, fontsize=11, fontweight='bold')
    ax.tick_params(axis="both", labelsize=10)
    ax.set_title(f"Panel a) {_display_name(var, all_vars)}", fontsize=15, pad=8, fontweight='bold')
    ax.grid(True, ls="--", alpha=.3)

# ----------------------------------------------------------------
# Electricity share - dots only
# ----------------------------------------------------------------
def plot_elec_share_ax_dots_only(ax, df_share, *, region,
                                  base_scenario, compare_scenarios,
                                  years, colours, m_mark,
                                  jitter=0.12):
    """Show all scenarios as dots"""
    years = [int(y) for y in years]
    df_r = df_share[df_share["region"] == region]
    
    if df_r.empty:
        ax.text(.5, .5, "No data", ha="center", va="center", fontsize=12)
        ax.axis("off"); return

    x = np.arange(len(years))
    
    # Plot all scenarios
    all_scenarios = [base_scenario] + list(compare_scenarios)
    
    for s_idx, scen in enumerate(all_scenarios):
        dfs = df_r[df_r["scenario"] == scen]
        if dfs.empty: continue
        
        for m_idx, model in enumerate(sorted(dfs["model"].unique())):
            dfm = dfs[(dfs["model"] == model) & (dfs["year"].isin(years))]
            if dfm.empty: continue
            
            offset = (-1)**s_idx * (jitter + m_idx*0.02) if scen != base_scenario else m_idx*0.02
            color = "black" if scen == base_scenario else colours.get(scen, "grey")
            
            ax.scatter(x + offset,
                       dfm.sort_values("year")["electricity_share"],
                       marker=m_mark[model], s=80, alpha=.8,
                       color=color,
                       edgecolors="dimgrey", linewidths=.5, zorder=3)

    ax.set_xticks(x)
    ax.set_xticklabels(years, fontsize=10)
    ax.set_ylabel("(%)", fontsize=11, fontweight='bold')
    ax.tick_params(axis="y", labelsize=10)
    ax.set_title("Panel b) Electricity Share in FE", fontsize=15, pad=8, fontweight='bold')
    ax.grid(True, ls="--", alpha=.3)

# ----------------------------------------------------------------
# Energy system cost - exclude baseline
# ----------------------------------------------------------------
def plot_cost_change_ax_no_baseline(ax, df_changes, *, region,
                                    variable, baseline_scenario,
                                    compare_scenarios, years,
                                    colours, m_mark, jitter=0.12):
    years = [int(y) for y in years]
    sub = df_changes[(df_changes["region"] == region) &
                     (df_changes["variable"] == variable) &
                     (df_changes["year"].isin(years)) &
                     (~df_changes["scenario"].isin([baseline_scenario]))]
    
    if sub.empty:
        ax.text(.5, .5, "No data", ha="center", va="center", fontsize=12)
        ax.axis("off"); return

    x = np.arange(len(years))
    ax.axhline(0, color="grey", ls="--", lw=1)

    for s_idx, scen in enumerate(compare_scenarios):
        dfs = sub[sub["scenario"] == scen]
        if dfs.empty: continue
        
        base_off = (-1)**s_idx * jitter
        for m_idx, model in enumerate(sorted(dfs["model"].unique())):
            dfm = dfs[dfs["model"] == model]
            offset = base_off + (-1)**m_idx*0.015
            ax.scatter(x + offset,
                       dfm.sort_values("year")["percentage_change"],
                       marker=m_mark[model], s=80, alpha=.8,
                       color=colours.get(scen,"black"),
                       edgecolors="dimgrey", linewidths=.5)

    ax.set_xticks(x)
    ax.set_xticklabels(years, fontsize=10)
    ax.set_ylabel("% Δ vs baseline", fontsize=11, fontweight='bold')
    ax.tick_params(axis="y", labelsize=10)
    ax.set_title("Panel d) Energy System Cost Change", fontsize=15, pad=8, fontweight='bold')
    ax.grid(True, ls="--", alpha=.3)

# ----------------------------------------------------------------
# Primary energy - improved visualisation
# ----------------------------------------------------------------
def plot_primary_energy_improved(ax, df_changes, *, region,
                                 variables, baseline_scenario,
                                 compare_scenarios, years,
                                 colours, m_mark, all_vars):
    """Improved primary energy panel with clear variable separation"""
    years = [int(y) for y in years]
    
    # Filter data
    sub = df_changes[(df_changes["region"] == region) &
                     (df_changes["variable"].isin(variables)) &
                     (df_changes["year"].isin(years)) &
                     (~df_changes["scenario"].isin([baseline_scenario]))]
    
    if sub.empty:
        ax.text(.5, .5, "No data", ha="center", va="center", fontsize=12)
        ax.axis("off"); return
    
    # Setup positions
    n_vars = len(variables)
    n_years = len(years)
    total_groups = n_vars * n_years
    
    # Create x positions for each variable-year combination
    x_positions = np.arange(total_groups)
    
    ax.axhline(0, color="grey", ls="--", lw=1)
    
    # Plot data
    for idx, (year, var) in enumerate([(y, v) for y in years for v in variables]):
        x_base = idx
        
        for s_idx, scen in enumerate(compare_scenarios):
            dfs = sub[(sub["scenario"] == scen) & 
                     (sub["variable"] == var) & 
                     (sub["year"] == year)]
            if dfs.empty: continue
            
            for m_idx, model in enumerate(sorted(dfs["model"].unique())):
                dfm = dfs[dfs["model"] == model]
                if not dfm.empty:
                    # Add jitter for visibility
                    jitter = (-1)**s_idx * 0.1 + (-1)**m_idx * 0.03
                    ax.scatter(x_base + jitter,
                              dfm["percentage_change"].values[0],
                              marker=m_mark[model], s=80, alpha=.8,
                              color=colours.get(scen, "black"),
                              edgecolors="dimgrey", linewidths=.5)
    
    # Add vertical separators between years
    for i in range(1, n_years):
        ax.axvline(x=i*n_vars - 0.5, color='lightgrey', ls='--', lw=0.8, alpha=0.5)
    
    # Custom x-axis labels
    var_labels = [_display_name(v, all_vars) for v in variables]
    tick_labels = []
    tick_positions = []
    
    for y_idx, year in enumerate(years):
        for v_idx, var_label in enumerate(var_labels):
            pos = y_idx * n_vars + v_idx
            tick_positions.append(pos)
            # Only show year for middle variable
            if v_idx == 1:
                tick_labels.append(f"{var_label}\n{year}")
            else:
                tick_labels.append(var_label)
    
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels, fontsize=9, rotation=0, ha='center')
    ax.set_ylabel("% Δ vs baseline", fontsize=11, fontweight='bold')
    ax.tick_params(axis="y", labelsize=10)
    ax.set_title("Panel c) Primary Energy Shift vs Baseline", fontsize=15, pad=8, fontweight='bold')
    ax.grid(True, axis='y', ls="--", alpha=.3)

# ----------------------------------------------------------------
# Alternative: Faceted primary energy panels
# ----------------------------------------------------------------
def plot_primary_energy_faceted(fig, gs_slot, df_changes, *, region,
                                variables, baseline_scenario,
                                compare_scenarios, years,
                                colours, m_mark, all_vars):
    """Create subpanels for each variable"""
    years = [int(y) for y in years]
    
    # Create sub-gridspec
    inner_gs = gs_slot.subgridspec(1, 3, wspace=0.15)
    
    # Filter data once
    sub = df_changes[(df_changes["region"] == region) &
                     (df_changes["variable"].isin(variables)) &
                     (df_changes["year"].isin(years)) &
                     (~df_changes["scenario"].isin([baseline_scenario]))]
    
    if sub.empty:
        ax = fig.add_subplot(gs_slot)
        ax.text(.5, .5, "No data", ha="center", va="center", fontsize=12)
        ax.axis("off")
        return
    
    # Create subplot for each variable
    for v_idx, var in enumerate(variables):
        ax = fig.add_subplot(inner_gs[v_idx])
        
        var_data = sub[sub["variable"] == var]
        x_pos = np.arange(len(years))
        
        ax.axhline(0, color="grey", ls="--", lw=0.8)
        
        for s_idx, scen in enumerate(compare_scenarios):
            dfs = var_data[var_data["scenario"] == scen]
            if dfs.empty: continue
            
            for m_idx, model in enumerate(sorted(dfs["model"].unique())):
                dfm = dfs[dfs["model"] == model]
                
                x_vals = []
                y_vals = []
                for y_idx, year in enumerate(years):
                    year_data = dfm[dfm["year"] == year]
                    if not year_data.empty:
                        jitter = (-1)**s_idx * 0.08 + (-1)**m_idx * 0.02
                        x_vals.append(x_pos[y_idx] + jitter)
                        y_vals.append(year_data["percentage_change"].values[0])
                
                if x_vals:
                    ax.scatter(x_vals, y_vals,
                              marker=m_mark[model], s=70, alpha=.8,
                              color=colours.get(scen, "black"),
                              edgecolors="dimgrey", linewidths=.5)
        
        ax.set_xticks(x_pos)
        ax.set_xticklabels(years, fontsize=9)
        ax.set_title(_display_name(var, all_vars), fontsize=11, pad=5)
        ax.grid(True, ls="--", alpha=.3)
        
        if v_idx == 0:
            ax.set_ylabel("% Δ vs baseline", fontsize=10)
        ax.tick_params(axis="both", labelsize=9)
    
    # Add overall title
    fig.text(0.5, 0.48, "Primary Energy Shift vs Baseline",
             ha='center', va='bottom', fontsize=15, fontweight='bold')

# ----------------------------------------------------------------
# Master builder
# ----------------------------------------------------------------
def create_daccs_dashboard_v4(*,
        df, df_changes, elec_share_df,
        region: str,
        years_bottom: List[int],
        scenarios: List[str],
        models: List[str],
        baseline_scenario: str,
        all_vars_names: Dict[str, str],
        colours: Dict[str, str],
        scenario_names: Dict[str, str],
        save_dir: str = "../figures/combined_panels/",
        figsize: tuple = (20, 12),
        share_years: Optional[List[int]] = None,
        cost_years: Optional[List[int]] = None,
        primary_style: str = "improved"):  # "improved" or "faceted"
    """
    Dashboard layout:
    Top:    [DACCS timeseries (2 cols)] [Electricity share (1 col)]
    Bottom: [Primary energy (2 cols)]    [Energy system cost (1 col)]
    """
    pathlib.Path(save_dir).mkdir(parents=True, exist_ok=True)

    scen_no_base = [s for s in scenarios if s != baseline_scenario]
    m_mark = _model_markers(models)

    fig = plt.figure(figsize=figsize)
    gs = gridspec.GridSpec(2, 3, figure=fig,
                          height_ratios=[1, 1], width_ratios=[1, 1, 1],
                          hspace=.35, wspace=.25)

    # ---- Top row ----
    # DACCS timeseries
    ax_daccs = fig.add_subplot(gs[0, :2])
    _ax_timeseries_daccs(df, ax=ax_daccs,
                        region=region, models=models, scenarios=scenarios,
                        all_vars=all_vars_names, colours=colours)

    # Electricity share
    ax_share = fig.add_subplot(gs[0, 2])
    plot_elec_share_ax_dots_only(ax_share, elec_share_df,
                                 region=region,
                                 base_scenario=baseline_scenario,
                                 compare_scenarios=scen_no_base,
                                 years=(share_years if share_years is not None else years_bottom),
                                 colours=colours, m_mark=m_mark)

    # ---- Bottom row ----
    # Primary Energy
    prim_vars = ["Primary Energy|Biomass",
                 "Primary Energy|Fossil",
                 "Primary Energy|Non-Biomass Renewables"]
    
    if primary_style == "faceted":
        plot_primary_energy_faceted(fig, gs[1, :2], df_changes,
                                   region=region,
                                   variables=prim_vars,
                                   baseline_scenario=baseline_scenario,
                                   compare_scenarios=scen_no_base,
                                   years=years_bottom,
                                   colours=colours, m_mark=m_mark,
                                   all_vars=all_vars_names)
    else:
        ax_primary = fig.add_subplot(gs[1, :2])
        plot_primary_energy_improved(ax_primary, df_changes,
                                    region=region,
                                    variables=prim_vars,
                                    baseline_scenario=baseline_scenario,
                                    compare_scenarios=scen_no_base,
                                    years=years_bottom,
                                    colours=colours, m_mark=m_mark,
                                    all_vars=all_vars_names)

    # Energy System Cost
    ax_cost = fig.add_subplot(gs[1, 2])
    plot_cost_change_ax_no_baseline(ax_cost, df_changes,
                                    region=region,
                                    variable="Energy System Cost",
                                    baseline_scenario=baseline_scenario,
                                    compare_scenarios=scen_no_base,
                                    years=(cost_years if cost_years is not None else years_bottom),
                                    colours=colours, m_mark=m_mark)

    # ---- Legend ----
    handles = []
    
    # Scenarios
    handles.append(mlines.Line2D([], [], color="black", marker="o", ls="None", ms=8,
                                label=scenario_names.get(baseline_scenario, baseline_scenario)))
    for s in scen_no_base:
        handles.append(mlines.Line2D([], [], color=colours[s], marker="o", ls="None", ms=8,
                                     label=scenario_names.get(s, s)))
    
    # Models
    for m in models:
        handles.append(mlines.Line2D([], [], color="black", marker=m_mark[m], ls="None", ms=8,
                                     label=m.replace("_", " ")))
    
    # Line styles
    for m in models:
        handles.append(mlines.Line2D([], [], color="black", ls=model_linestyles.get(m, "-"), lw=2.5,
                                     label=f"{m.replace('_', ' ')}"))

    fig.legend(handles=handles,
               ncol=len(handles),
               bbox_to_anchor=(.5, .02), loc="lower center",
               fontsize=9, frameon=True,
               title="Scenario                      |                     Model marker                     |                     Model line-style")

    # ---- Title & save ----
    title = "DACCS Illustrative Scenarios" if region == "World" else f"{region} – DACCS Illustrative Scenarios"
    fig.suptitle(title, fontsize=18, fontweight="bold", y=.975)

    fig.tight_layout(rect=[0, .07, 1, .94])
    
    # Save without metadata
    fname = f"{save_dir}/dashboard_daccs_v4_{region.replace(' ', '_')}.png"
    fig.savefig(fname, dpi=150, bbox_inches="tight")
    
    # PDF without metadata
    pdf_fname = fname.replace('.png', '.pdf')
    fig.savefig(pdf_fname, dpi=150, bbox_inches="tight",
                metadata={'Creator': '', 'Producer': '', 'CreationDate': None})
    
    plt.close(fig)
    print(f"Saved: {fname}")

In [62]:
create_daccs_dashboard_v4(
    df=df,
    df_changes=df_with_changes,
    elec_share_df=merged,
    region="World",
    years_bottom=[2040, 2070, 2100],
    share_years=[2040, 2070, 2100],
    cost_years=[2030, 2040, 2050, 2070, 2100],
    scenarios=scenarios_of_interest,
    models=models_of_interest,
    baseline_scenario="D2_NDC_DACCS_2_4",
    all_vars_names=all_vars_names,
    colours=colours,
    scenario_names=scenario_names,
    primary_style="improved"  # Clear labels under each group
)



Saved: ../figures/combined_panels//dashboard_daccs_v4_World.png


In [63]:
# # Option 2: Faceted style with separate subpanels for each variable
# create_daccs_dashboard_v4(
#     df=df,
#     df_changes=df_with_changes,
#     elec_share_df=merged,
#     region="World",
#     years_bottom=[2040, 2070, 2100],
#     share_years=[2040, 2070, 2100],
#     cost_years=[2030, 2040, 2050, 2070, 2100],
#     scenarios=scenarios_of_interest,
#     models=models_of_interest,
#     baseline_scenario="D2_NDC_DACCS_2_4",
#     all_vars_names=all_vars_names,
#     colours=colours,
#     scenario_names=scenario_names,
#     primary_style="faceted"  # Three separate mini-panels
# )